In [16]:
import importlib
import blizzard_api
import mysql
import mplusdb
import time
import sys

In [33]:
t0 = time.time()
mdb = mplusdb.MplusDatabase('.db_config')
conn = mdb.connect()
cursor = conn.cursor()
cursor.execute('SELECT id, level, dungeon, duration FROM new_table where score is null')
data = cursor.fetchall()
#conn.commit()
cursor.close()
conn.close()

print(time.time() - t0)

7.357625722885132


In [27]:
conn = mdb.connect()
cursor = conn.cursor()
cursor.execute('SELECT * from dungeon')
dungeon = cursor.fetchall()
#conn.commit()
cursor.close()
conn.close()

In [28]:
dungeon
ids = [i[0] for i in dungeon]
timers = [i[-1] for i in dungeon]

dungeon_timer = dict(zip(ids, timers))

In [29]:
class Scorer:
    def __init__(self, dungeon_timers):
        self.dungeon_timers = dungeon_timers
    
    def get_ratio_user_vs_base_timer(self, user_time, user_dungeon):
        default_time = self.dungeon_timers[user_dungeon]
        return user_time / default_time
    
    def get_score(self, user_time, user_dungeon, dungeon_level):
        ratio = self.get_ratio_user_vs_base_timer(user_time, user_dungeon)
    
        base_points = self.get_base_score(dungeon_level)
        final_score = 0
        if ratio <= 1.0: # successful timer
            bonus = self.get_bonus_points(ratio, base_points)
            final_score = base_points + bonus
        else:
            penalty = self.get_penalty_points(ratio, base_points)
            final_score = base_points - penalty
        if final_score < 10:
            final_score = 10
        return final_score
    
    def get_bonus_points(self, ratio, base_points):
        flat_bonus = (1 - ratio) * 0.085
        chest_bonus = 0
        if ratio <= 0.6:
            chest_bonus = 0.03
        elif ratio <= 0.8:
            chest_bonus = 0.015
        pct_bonus = flat_bonus + chest_bonus
        bonus_points = base_points * pct_bonus
        return bonus_points
            
    def get_penalty_points(self, ratio, base_points):
        base_penatly_points = base_points * 0.1
        additional_penalty_points = (ratio - 1) * (0.24 * base_points * 0.9)
        total_penalty = base_penatly_points + additional_penalty_points
        return total_penalty
    
    def get_base_score(self, dungeon_level):
        """Calculates score for key level."""
        if dungeon_level <= 10:
            return 10*dungeon_level
        else:
            return 100 * 1.1**(dungeon_level - 10)
    
    
scorer = Scorer(dungeon_timer)

print(scorer.get_score(2577000, 248, 15))
print('---')
print(scorer.get_score(3210000, 248, 15))

141.4225996615385
---
132.0122658461539


In [34]:
print(len(data))

2316121


In [35]:
t0 = time.time()
conn = mdb.connect()
cursor = conn.cursor()
i = 0
for record in data:
    id_, level, dungeon, run_time = record
    score = scorer.get_score(run_time, dungeon, level)
    query = 'UPDATE new_table set score = %f where id = %d' % (score, id_)
    cursor.execute(query)
    conn.commit()
    print('\r%s' % i, end='')
    i += 1


cursor.close()
conn.close()
print(time.time() - t0)

231612013001.866752147675


In [32]:
cursor.close()
conn.close()